# Tensorflow Multi-Layer Perceptron with iris dataset

Simple Multi-Layer Perceptron fro the iris dataset

In [53]:
# Load our data

import tensorflow as tf
import numpy as np
import pandas as pd

import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

iris_data = load_iris()

IRIS_TRAINING = '../data/iris/iris_training.csv'
IRIS_TEST = '../data/iris/iris_test.csv'

training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)

X_train = training_set.data
y_train = training_set.target
X_test = test_set.data
y_test = test_set.target

# Convert to one-hot encoding for labels
label_binarizer = sklearn.preprocessing.LabelBinarizer()
label_binarizer.fit(range(max(y_train)+1))
y_train = label_binarizer.transform(y_train)
y_test = label_binarizer.transform(y_test)

In [54]:
#  define a get next_batch function
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [55]:
# Define Params

# Parameters
learning_rate = 0.1
num_steps = 1000
batch_size = 10
display_step = 10

# Network Parameters
n_hidden_1 = 20 # 1st layer number of neurons
n_hidden_2 = 20 # 2nd layer number of neurons
num_input = 4
num_classes = 3

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [56]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [57]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [58]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [59]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(tf.global_variables_initializer())

    for step in range(1, num_steps+1):
        batch_x, batch_y = next_batch(batch_size, X_train, y_train)
        
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: X_test,
                                      Y: y_test}))

Step 1, Minibatch Loss= 285.5261, Training Accuracy= 0.100
Step 10, Minibatch Loss= 19.8424, Training Accuracy= 0.700
Step 20, Minibatch Loss= 2.3154, Training Accuracy= 0.700
Step 30, Minibatch Loss= 0.5173, Training Accuracy= 0.900
Step 40, Minibatch Loss= 1.3769, Training Accuracy= 0.800
Step 50, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 60, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 70, Minibatch Loss= 0.0050, Training Accuracy= 1.000
Step 80, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 90, Minibatch Loss= 0.5275, Training Accuracy= 0.900
Step 100, Minibatch Loss= 0.0022, Training Accuracy= 1.000
Step 110, Minibatch Loss= 0.0000, Training Accuracy= 1.000
Step 120, Minibatch Loss= 1.0102, Training Accuracy= 0.800
Step 130, Minibatch Loss= 0.1304, Training Accuracy= 0.900
Step 140, Minibatch Loss= 0.1875, Training Accuracy= 0.900
Step 150, Minibatch Loss= 0.0001, Training Accuracy= 1.000
Step 160, Minibatch Loss= 0.0120, Training Accuracy= 1.000
Step 